<a href="https://colab.research.google.com/github/NTT123/TacotronS/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!sudo apt install sox
!pip install -Uqq unidecode pax3 opax toml fire tensorflow-cpu

In [2]:
%cd /content
!git clone https://github.com/NTT123/TacotronS.git
%cd /content/TacotronS

/content
Cloning into 'TacotronS'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 212 (delta 1), reused 8 (delta 1), pack-reused 204
Receiving objects: 100% (212/212), 48.00 MiB | 26.21 MiB/s, done.
Resolving deltas: 100% (138/138), done.
/content/TacotronS


In [3]:
import os
from pathlib import Path
from tqdm.cli import tqdm

In [4]:
# download LJSpeech dataset
!python ljs.py

100%|█████████████████████████████████████| 2.75G/2.75G [00:00<00:00, 13.3TB/s]
Untarring contents of '/root/.cache/pooch/b5107092215abf6367ef755eb0633116-LJSpeech-1.1.tar.bz2' to '/root/.cache/pooch/b5107092215abf6367ef755eb0633116-LJSpeech-1.1.tar.bz2.untar'
Data is prepared in the directory '/root/.cache/pooch/b5107092215abf6367ef755eb0633116-LJSpeech-1.1.tar.bz2.untar/LJSpeech-1.1/wavs'.

Run 'python data.py /root/.cache/pooch/b5107092215abf6367ef755eb0633116-LJSpeech-1.1.tar.bz2.untar/LJSpeech-1.1/wavs' to create the tensorflow dataset.


In [5]:
# resample to 24k
!mkdir -p wav_24k
wav_dir = Path("/root/.cache/pooch/b5107092215abf6367ef755eb0633116-LJSpeech-1.1.tar.bz2.untar/LJSpeech-1.1/wavs")
wav_files = sorted(wav_dir.glob("*.wav"))
for path in tqdm(wav_files):
    cmd = f"sox {path} -c 1 --norm=-3 -r 24000 wav_24k/{path.name}"
    os.system(cmd)

100%|██████████| 13100/13100 [04:19<00:00, 50.49it/s]


In [6]:
!cp {wav_dir}/*.txt ./wav_24k

In [7]:
# create tf dataset
!python data.py wav_24k

Loading data from directory 'wav_24k'
100% 13100/13100 [00:00<00:00, 15091.60it/s]
100% 13100/13100 [03:42<00:00, 58.85it/s] 
Created a tensorflow dataset at 'tf_data'.

Run 'python train.py' to train your model.


In [8]:
# uncomment to save checkpoints to google drive
# !mkdir /content/drive/MyDrive/{logs,ckpts}
# !ln -sfT /content/drive/MyDrive/ckpts ./ckpts
# !ln -sfT /content/drive/MyDrive/logs ./logs
!mkdir -p ckpts logs

In [ ]:
# Use reduction factor of 2 for robust attention learning
!RR=2 TRAINING_STEPS=50000 python train.py --batch-size=64 | tee -a ./logs/training_logs.txt

2022-04-25 11:22:59.032696: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
Devices: [GpuDevice(id=0, process_index=0)]
step 0000000  loss 0.269  LR 1.024e-03  41.03s
step 0000100  loss 1.137  LR 1.024e-03  94.76s
step 0000200  loss 0.871  LR 1.024e-03  88.59s
step 0000300  loss 0.802  LR 1.024e-03  88.53s
step 0000400  loss 0.766  LR 1.024e-03  88.73s
step 0000500  loss 0.741  LR 1.024e-03  88.48s
step 0000600  loss 0.718  LR 1.024e-03  88.53s
step 0000700  loss 0.698  LR 1.024e-03  88.58s
step 0000800  loss 0.686  LR 1.024e-03  88.51s
step 0000900  loss 0.668  LR 1.024e-03  88.46s
step 0001000  loss 0.657  LR 1.024e-03  88.49s


In [ ]:
!python train.py --batch-size=32 | tee -a ./logs/training_logs.txt